Step 1: Loading the libraries and setting the height and width of the graph

In [ ]:
#importing the libraries
#  Visualization
library(ggplot2)

# Data manipulation
library(tibble)
library(dplyr)
library(tidyr)
library(purrr)
library(readr)

# Modelling and Preprocessing
library(tidymodels)

# Data splitting and resampling
library(rsample) # train_test_split_equivalent
library(tune) # GridSearchCV equivalent
library(workflows) #make_pipeline equivalent
library(recipes) #make_column_transformer and StandardScaler equivalent

# Metrics
library(yardstick) # for mean squared error

#Time and Dates
library(lubridate)


#formatting graphs
options(repr.plot.width=12, repr.plot.height=6)

Step 2: Loading the data into R

In [ ]:
#loading data into R
players<-read_csv("players.csv")
players
sessions<- read_csv("sessions.csv")
sessions

In [ ]:
# CONVERT START_TIME AND END_TIME TO DATETIME
sessions<-sessions|>
mutate( start_time= dmy_hm(start_time), end_time= dmy_hm(end_time))
sessions


In [ ]:
#CONVERT THE TIMEZONE FROM GMT TO PST
sessions <- sessions |>
  mutate(
    start_time = with_tz(force_tz(start_time, tzone = "GMT"), tzone = "America/Los_Angeles"),
    end_time = with_tz(force_tz(end_time, tzone = "GMT"), tzone = "America/Los_Angeles")
  )
sessions 


WRANGLING DATA

In [ ]:
#Creating an hourly time range
hours<- seq(from= floor_date(min(sessions$start_time, na.rm = TRUE), unit= "hour"), 
            to = ceiling_date(max(sessions$end_time, na.rm = TRUE), unit= "hour"), 
            by="1 hour")
head(hours)


In [ ]:
#Count how many sessions are active during each hour
concurrent_sessions<-map_dfr(hours, function(h){
active_players <- sum(sessions$start_time <= h & sessions$end_time >= h)
tibble( time=h, active_sessions = active_players)})
head(concurrent_sessions)

In [ ]:
#Extracting day of week
concurrent_sessions <- concurrent_sessions|>
mutate(day_of_week =wday(time, label = TRUE, abbr = FALSE))
concurrent_sessions


In [ ]:
# Create weekday sessions
weekday_sessions <-concurrent_sessions|>
filter(day_of_week %in% c ("Monday", "Tuesday", "Wednesday", "Thursday", "Friday"))|>
mutate( day_of_end ="weekday")
head(weekday_sessions)

In [ ]:
# Create weekend sessions
weekend_sessions <- concurrent_sessions |>
  filter(day_of_week %in% c("Saturday", "Sunday")) |>
  mutate(day_or_end = "weekend")
head( weekend_sessions)

In [ ]:
# Calculate sessions duration 
sessions_durations <- sessions |>
  mutate(
    play_duration_minutes = as.numeric(difftime(end_time, start_time, units = "mins")),
    day_of_week = wday(start_time, label = TRUE, abbr = FALSE),
    start_hour = hour(start_time),
    start_time_of_day = hour(start_time) + minute(start_time) / 60
  ) |>
  filter(play_duration_minutes > 0)
head(sessions_durations)

In [ ]:
#Show concurrent sessions by hour and the plot

concurrent_sessions<- concurrent_sessions|>
mutate(hour= hour(time))

total_sessions_plot<- ggplot(concurrent_sessions, aes(x= factor(hour), y= active_sessions))+
geom_bar(stat= "identity", fill="skyblue")+
labs(
title="Total Actve sessions by hour",
x="Hour of day",
y="Total Active sessions")+
theme_minimal()
total_sessions_plot


In [ ]:
# Day of the week exploratory plot

concurrent_sessions<- concurrent_sessions|>
mutate( day_of_week= factor (day_of_week , levels= c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")))
                                                      
dayofweek_exploratory_plot<- ggplot(concurrent_sessions, aes(x= day_of_week, y= active_sessions))+
geom_bar(stat= "identity", fill="green")+
labs(
title="Total sessions by the day of week",
x="DAY OF WEEK",
y="Total Active sessions")+
theme_minimal()
dayofweek_exploratory_plot


In [ ]:
#Show concurrent sessions by hour and day of week

concurrent_sessions<- concurrent_sessions|>
mutate(
hour=hour(time),
day_of_week= factor (day_of_week , levels= c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")))

Total_sessions_dayofweek <- ggplot(concurrent_sessions, aes(x=factor(hour), y= active_sessions))+
geom_bar(stat= "identity", fill="pink")+
facet_wrap(~ day_of_week, ncol=1)+
labs(
title="Total sessions by the by hour by day",
x="hour of day",
y="Total Active sessions")+
theme_minimal()
Total_sessions_dayofweek



In [ ]:
#Scatter plot of start_time_of_day vs. play_duration_minutes
sessions_durations <-sessions_durations |>
mutate(
 day_of_week= factor (day_of_week , levels= c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")))
head(sessions_durations)

duration_scatter <- ggplot(sessions_durations , aes(x=start_time_of_day , y= play_duration_minutes ))+
geom_point( alpha= 0.4, color="red")+
facet_wrap(~ day_of_week, ncol=1)+
labs(
title="session duration in minutes by the time of day",
x="hour of day",
y="duration of session (minutes)")+
theme_minimal()
duration_scatter

In [ ]:
#Bar chart play duration minutes vs day of the week 
avg_duration <- sessions_durations|>
group_by(day_of_week)|>
summarize(avg_play_duration = mean ( play_duration_minutes, na.rm= TRUE))

duration_bar <- ggplot(avg_duration , aes(x= day_of_week  , y= avg_play_duration))+
geom_col( fill="purple")+
labs(
title="session duration in minutes by day of the week",
x="day of the week",
y="duration of session (minutes)")+
theme_minimal()
duration_bar



ANALYSIS OF HOURS REGARDLESS OF WEEK